In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
import re
import yaml
from sariModelInterpreter import modelInterpreter as mi

In [3]:
inputFile = '../models/bso.yml'

In [4]:
model = mi.parseModelFromFile(inputFile)

In [5]:
def getNodeWithId(model, id):
    
    def traverseNode(id, node):
        if node['id'] == id:
            nodeToReturn.append(node)
        elif 'children' in node:
            for child in node['children']:
                traverseNode(id, child)
    
    nodeToReturn = []
    for node in model:
        traverseNode(id, node)
    
    if len(nodeToReturn):
        return nodeToReturn[0]
    else:
        return False

def convertModelToGraph(model):
    def fillGraph(node):
        if 'children' in node:
            graph[node['id']] = [d['id'] for d in node['children']]
            for child in node['children']:
                fillGraph(child)
        else:
            graph[node['id']] = []   
          
    graph = {}  
    for node in model:
        fillGraph(node)
        
    return graph

def findPath(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return path
    if not start in graph:
        return None
    for node in graph[start]:
        if node not in path:
            newPath = findPath(graph, node, end, path)
            if newPath:
                return newPath
    return None
    
    
def compileQueryForNodes(model, rootId, nodeIds):
    root = getNodeWithId(model, rootId)
    graph = convertModelToGraph(model)
    paths = []
    for nodeId in nodeIds:
        paths.append(findPath(graph, rootId, nodeId))
    return paths
    
compileQueryForNodes(model, 'artwork',['work_creator', 'artwork_medium'])


[['artwork', 'work', 'work_creation', 'work_creator'],
 ['artwork', 'artwork_medium']]